In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\aryan\mediapipe_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\aryan\mediapipe_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install screen-brightness-control

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\aryan\mediapipe_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
pip install pycaw

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\aryan\mediapipe_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install comtypes

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\aryan\mediapipe_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [10]:
import sys

In [11]:
print('sys.version')

sys.version


In [12]:
print(sys.version)

3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]


In [13]:
import cv2
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from google.protobuf.json_format import MessageToDict

In [16]:

# Initialize MediaPipe Hand module
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)
mpDraw = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize audio device for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# Get volume range
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]  # Usually -65.25 dB
max_vol = vol_range[1]  # Usually 0.0 dB

while True:
    # Read frame from webcam
    success, frame = cap.read()
    if not success:
        break
        
    # Define region of interest (ROI)
    pt1 = (300, 180)
    pt2 = (550, 420)
    
    # Draw rectangle for ROI
    cv2.rectangle(frame, pt1, pt2, (255, 0, 0), 1)
    
    # Crop the frame to ROI
    img = frame[pt1[1]:pt2[1], pt1[0]:pt2[0]]
    
    # Flip the image horizontally
    img = cv2.flip(img, 1)
    
    # Convert to RGB for MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the image with MediaPipe Hands
    results = hands.process(imgRGB)
    
    # Initialize landmark list
    lmList = []
    
    # Check if hands are detected
    if results.multi_hand_landmarks:
        # If two hands are detected
        if len(results.multi_handedness) == 2:
            cv2.putText(img, 'Both Hands', (20, 50),
                      cv2.FONT_HERSHEY_COMPLEX,
                      0.9, (0, 255, 0), 2)
        else:
            # Identify if it's left or right hand
            for i in results.multi_handedness:
                label = MessageToDict(i)['classification'][0]['label']
                
                if label == 'Left':  # This will appear as right on screen due to mirroring
                    # Volume control with right hand (appears as left in mirrored image)
                    cv2.putText(img, 'Right Hand (Volume)', (20, 50),
                             cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)
                    
                elif label == 'Right':  # This will appear as left on screen due to mirroring
                    # Brightness control with left hand (appears as right in mirrored image)
                    cv2.putText(img, 'Left Hand (Brightness)', (20, 50),
                              cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)
                
        # Process hand landmarks
        for handLms in results.multi_hand_landmarks:
            # Extract landmark coordinates
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
            
            # Draw landmarks on the image
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
        
        # If landmarks are detected
        if lmList:
            # Get coordinates of thumb tip (4) and index finger tip (8)
            thumb_tip_x, thumb_tip_y = lmList[4][1], lmList[4][2]
            index_tip_x, index_tip_y = lmList[8][1], lmList[8][2]
            
            # Draw circles at thumb and index fingertips
            cv2.circle(img, (thumb_tip_x, thumb_tip_y), 10, (255, 0, 0), cv2.FILLED)
            cv2.circle(img, (index_tip_x, index_tip_y), 10, (255, 0, 0), cv2.FILLED)
            
            # Draw line between thumb and index finger
            cv2.line(img, (thumb_tip_x, thumb_tip_y), (index_tip_x, index_tip_y), (255, 0, 0), 3)
            
            # Calculate distance between thumb and index finger
            length = hypot(index_tip_x - thumb_tip_x, index_tip_y - thumb_tip_y)
            
            # Check for hand label to control volume or brightness
            if len(results.multi_handedness) == 1:
                label = MessageToDict(results.multi_handedness[0])['classification'][0]['label']
                
                if label == 'Left':  # Right hand in mirrored image - Volume Control
                    # Map finger distance to volume range
                    vol = np.interp(length, [30, 200], [min_vol, max_vol])
                    
                    # Set volume
                    volume.SetMasterVolumeLevel(vol, None)
                    
                    # Display volume level
                    vol_percentage = int(np.interp(vol, [min_vol, max_vol], [0, 100]))
                    cv2.putText(img, f'Vol Set: {vol_percentage}', (20, 150), 
                             cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)
                
                elif label == 'Right':  # Left hand in mirrored image - Brightness Control
                    # Map finger distance to brightness range (0-100)
                    brightness = np.interp(length, [30, 200], [0, 100])
                    
                    # Set brightness
                    sbc.set_brightness(int(brightness))
                    
                    # Display brightness level
                    cv2.putText(img, f'Bri Set: {int(brightness)}', (20, 150), 
                              cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the hand tracking frame
    cv2.imshow('Hand Gesture Control', img)
    
    # Full frame for context
    cv2.imshow('Full Frame', frame)
    
    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [18]:
# In a notebook cell, run:
import os
print(os.getcwd())  # Current working directory

C:\Users\aryan\Downloads
